In [ ]:
import tensorflow as tf
import numpy as np
import os

In [2]:
import keras
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
# now with image augmentation
from keras.preprocessing.image import ImageDataGenerator
os.environ["CUDA_VISIBLE_DEVICES"]="1"

Using TensorFlow backend.


In [3]:
# data is saved in ~/.keras/datasets/cifar-10-batches-py/
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [4]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((50000, 32, 32, 3), (50000, 1), (10000, 32, 32, 3), (10000, 1))

In [5]:
num_classes = len(np.unique(y_train))
num_classes

10

In [6]:
np.unique(y_train)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=uint8)

In [7]:
y_train[0][0]

6

In [8]:
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [9]:
y_train[0]

array([0., 0., 0., 0., 0., 0., 1., 0., 0., 0.], dtype=float32)

In [10]:
y_test[0]

array([0., 0., 0., 1., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [11]:
def create_model(input_shape, num_classes):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), padding='same',
                     input_shape=input_shape))
    model.add(Activation('relu'))
    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes))
    model.add(Activation('softmax'))
    return model

In [12]:
input_shape = x_train.shape[1:]
model = create_model(input_shape, num_classes)

In [13]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 30, 30, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 30, 30, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 15, 15, 64)        18496     
__________

In [14]:
def create_optimizer(lr=1e-4, decay=1e-6):
    opt = keras.optimizers.rmsprop(lr=lr, decay=decay)
    return opt

In [15]:
opt = create_optimizer()
model.compile(loss='categorical_crossentropy', 
              optimizer=opt, 
              metrics=['accuracy'])

In [16]:
def convert_rgb_to_float(x):
    """
    convert rgb, 0-255 to floats: 0.-1.
    """
    if 'float' in str(x.dtype):
        pass
    else:
        x = x.astype('float32')
        x /= 255
    return x

In [17]:
x_train = convert_rgb_to_float(x_train)
x_test = convert_rgb_to_float(x_test)

In [18]:
def fit_model(model, x, y,
              validation_data=None,
              batch_size=32, 
              epochs=100, 
              shuffle=True,
              data_augmentation=False
             ):
    if not data_augmentation:
        model.fit(x, y, 
              batch_size=batch_size,
              epochs=epochs,
              validation_data=validation_data,
              shuffle=shuffle
             )
    else:
        datagen = ImageDataGenerator(
            featurewise_center=False,  # set input mean to 0 over the dataset
            samplewise_center=False,  # set each sample mean to 0
            featurewise_std_normalization=False,  # divide inputs by std of the dataset
            samplewise_std_normalization=False,  # divide each input by its std
            zca_whitening=False,  # apply ZCA whitening
            rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
            width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
            height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
            horizontal_flip=True,  # randomly flip images
            vertical_flip=False # randomly flip images   
        )
        datagen.fit(x)
        model.fit_generator(datagen.flow(x, y, batch_size=batch_size),
                           epochs=epochs,
                           validation_data=validation_data,
                           workers=1)
        
    return model

In [19]:
def save_model(model, save_dir, model_name):
    if not os.path.isdir(save_dir):
        os.makedirs(save_dir)
    model_path = os.path.join(save_dir, model_name)
    model.save(model_path)
    print('Saved trained model at %s ' % model_path)

In [ ]:
if False:
    model = fit_model(model, x_train, y_train, 
                      validation_data=(x_test, y_test), 
                      data_augmentation=False)
    model_name = 'keras_cifar10_trained_model.h5'
    save_dir = os.path.join(os.getcwd(), 'saved_models')
    save_model(model, save_dir, model_name)
    scores = model.evaluate(x_test, y_test, verbose=1)
    print('Test loss:', scores[0])
    print('Test accuracy:', scores[1])

In [22]:
"""
Epoch 100/100
50000/50000 [==============================] - 17s 345us/step 
- loss: 0.6178 - acc: 0.7955 
- val_loss: 0.7984 - val_acc: 0.7486

Test loss: 0.7984250497817993
Test accuracy: 0.7486
"""

'\nEpoch 100/100\n50000/50000 [==============================] - 17s 345us/step - \nloss: 0.6178 - acc: 0.7955 - val_loss: 0.7984 - val_acc: 0.7486\n\nTest loss: 0.7984250497817993\nTest accuracy: 0.7486\n'